In [11]:
# IR PS3
import re
import string

In [12]:
f_time_all = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.ALL", 'r')
f_query = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.QUE", 'r')
f_stp = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.STP", 'r')

In [13]:
def parse_documents(f):
    documents = []
    fstr = f.read()
    regex = r"\*TEXT\s+\d{3}\s+\d{2}/\d{2}/\d{2}\s+PAGE\s+\d{3}\n\n"
    
    iters = []
    result = re.finditer(regex, fstr, re.DOTALL)
    for key in result:
        iters.append(key)

    for i in range(len(iters)-1):
        i1 = iters[i].span()[1]
        i2 = iters[i+1].span()[0]
        
        documents.append(fstr[i1:i2])
    
    return documents

In [14]:
def parse_queries(f):
    text = f.read()
    result = re.findall(r'FIND\s+\d+\s+(.*)', text)
    
    return result

In [15]:
def parse_stopwords(f):
    text = f.read()
    text = text.lower()
    
    return set(nltk.word_tokenize(text))

In [16]:
docs = parse_documents(f_time_all)

In [17]:
queries = parse_queries(f_query)

In [18]:
samp_doc = docs[0]

In [19]:
stopwords = parse_stopwords(f_stp)

In [20]:
# Preprocessing
# 1. conv. to lowercase
# 2. tokenization
# 3. stopword, spl char removal
# 4. stemming

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TEMP.CS2K16.001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TEMP.CS2K16.001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [23]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords and i not in string.punctuation:
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [24]:
transform_text(samp_doc)[:251]

'alli nassau decemb 1960 propos help nato develop nuclear strike forc europ attempt devis plan studi nassau accord presid kennedi prime minist macmillan european saw emerg outlin nuclear nato want support sprang crisi cancel skybolt missil offer suppli'

In [25]:
processed_docs = []

for doc in docs:
    processed_docs.append(transform_text(doc))

In [26]:
# 1. Cosine Similarity

In [27]:
# Wik = (TFik/LENi)*log(N+1)/0.5+DFk
# TFik - term frequency within doc.
# LENi - len of current doc.
# N - total docs
# DFk - document frequency of the word (no. of documents for which the word exists)

In [28]:
import math

In [29]:
def TF(word, mymap):
    return mymap[word]

def DF(word, corpus_maps):
    count = 0
    
    for hm in corpus_maps:
        if word in hm:
            count += 1
    
    return count

def calculate_weights(doc, doc_map, corpus_maps):
    vector = [0]*len(doc_map)
    
    mymap = {}
    
    for i in doc.split():
        if i in mymap:
            mymap[i] +=1
        else:
            mymap[i] = 1
    
    for word in doc.split():
        TFik = TF(word, mymap)
        LENi = len(doc)
        N = len(corpus_maps)
        DFk = DF(word, corpus_maps)
        
        try:
            index = list(doc_map.keys()).index(word)
            vector[index] = (TFik/LENi)*(math.log(N+1)/(0.5+DFk))
        except ValueError:
            pass
    
    return vector

In [30]:
doc_map = {}

for doc in processed_docs:
    for word in doc.split():
        if word in doc_map:
            doc_map[word] += 1
        else:
            doc_map[word] = 1

doc_matrix = []

list_of_maps = []

for doc in processed_docs:
    hm = {}
    
    for word in doc.split():
        if word in hm:
            hm[word] += 1
        else:
            hm[word] = 1
    
    list_of_maps.append(hm)

for doc in processed_docs:
    vector = calculate_weights(doc, doc_map, list_of_maps)
    doc_matrix.append(vector)

In [33]:
print(doc_matrix[:2])

[[0.00019525287934412622, 0.0011324667001959322, 0.00014643965950809467, 5.5152599035516175e-05, 4.044523929271186e-05, 0.00010295151819963019, 0.0003069939849928731, 0.00020971605559183927, 0.00048813219836031554, 9.333516759856584e-05, 0.00013147833206609118, 9.165647753384342e-05, 4.2896465916512576e-05, 0.00047186112508163837, 2.01267778470841e-05, 6.961885452024173e-05, 5.5152599035516175e-05, 2.9594077531252582e-05, 0.0001866703351971317, 5.2107363505947796e-05, 2.520326484115576e-05, 0.00029801755268314, 6.291481667755178e-05, 9.437222501632767e-05, 4.2896465916512576e-05, 0.00028311667504898305, 2.0715854271876807e-05, 2.1778205772998695e-05, 0.0014155833752449152, 4.566397984661016e-05, 9.876163083104059e-05, 0.0011582045797458396, 0.00016653922061704883, 0.00022955406085052677, 8.409406189573753e-05, 0.0002348433249254237, 0.000223513164512355, 0.0009589435767788135, 0.00022351316451235502, 0.000273983879079661, 1.4494027732883092e-05, 0.0014155833752449152, 6.154710327151804